In [1]:
import pandas as pd 

In [2]:
stores = pd.read_csv("stores.csv")

### Working with dates

Many times the data can have date-time information. Pandas will treat date columns as strings. Once date columns are appropriately handled, meta data such as dayofweek, month, year, quarter etc can be extracted.

In [3]:
stores['Order Date'].head()

0    11-09-2013
1    11-09-2013
2     6/13/2013
3    10-11-2012
4    10-11-2012
Name: Order Date, dtype: object

In [4]:
stores['Order Date'] = pd.to_datetime(stores['Order Date']) 

In [5]:
stores['Ship Date'] = pd.to_datetime(stores['Ship Date'])

In [6]:
stores['Order Date'].dt.weekday.head()

0    5
1    5
2    3
3    3
4    3
Name: Order Date, dtype: int64

In [7]:
stores['Order Date'].dt.month.head()

0    11
1    11
2     6
3    10
4    10
Name: Order Date, dtype: int64

In [8]:
stores['Order Date'].dt.year.head()

0    2013
1    2013
2    2013
3    2012
4    2012
Name: Order Date, dtype: int64

### Class Excercise: Flight Delays Dataset

**Load the data set FlightDelays, the data has information on the flights over the year 2004 and if a particular flight was delayed or not.**
1. Find out the number of delayed flights for all weekdays
2. Find the average distance, total distance and count for all delayed flights on Friday.
3. Find out how many flights were on time on Week days and Weekends (Consider Saturday and Sunday as weekends)
4. Find out the number of flights for each destination across all weekdays
5. Find out the number of times weather was bad across all weekdays. (1 indicates bad weather)



### Map and Apply Constructs

Apply and map constructs are used when we want to loop over rows, rows+columns. `maps()` are used only to loop over rows of one column. `apply()` is used to loop over rows across columns or columns across rows.

Since a dataframe is two dimensional data-structure. We can specify in which direction an `apply()` operation can be applied. Pandas uses the idea of axes to help in specifying the direction of operation. Below is a gif that gives an intuitive idea about the axes/axis
![](axis.gif)

#### Map

Lets imagine we wanted to bucket the sales column into high,medium and low sales based on the following rule:
- If sales>5000, high
- If 5000<sales<=2000, medium
- If sales<=2000, low

we can use the notion of map to accomplish this.

In [9]:
def categorise(val):
    if val>5000:
        return 'high'
    elif val<=5000 and val>2000:
        return 'medium'
    else:
        return 'low'
stores['Sales'].map(categorise)

0       low
1       low
2       low
3       low
4       low
       ... 
9989    low
9990    low
9991    low
9992    low
9993    low
Name: Sales, Length: 9994, dtype: object

In [10]:
stores['Sales'].map(categorise).value_counts()

low       9854
medium     121
high        19
Name: Sales, dtype: int64

### Class Excercise: Comey Dataset

Use the dataset names `comey.csv`. You need to use the idea of map and find out the length of each response and each question. The you need to see if there is any difference between the lengths by party affiliattion. You can read more about the testimony of James Comey (former FBI director) [here](https://www.intelligence.senate.gov/sites/default/files/documents/os-jcomey-060817.pdf?)

### Apply

Apply is used to traverse rows across columns or vice-versa. Suppose in the stores dataset we want to do a rebalancing of sales figures. Imagine the charge of first class shipping was recognised incorrectly and now we need to decrease the sales number by $ 200. We can make use of the notion of apply here. In this case we will write a function that will traverse a row across columns, hence the correct axis direction would be 1

In [11]:
def rebalance(row):
    if row['Ship Mode']=='First Class':
        return row['Sales'] - 200
    else:
        return row['Sales']
stores.apply(rebalance,axis=1)

0       261.9600
1       731.9400
2        14.6200
3       957.5775
4        22.3680
          ...   
9989     25.2480
9990     91.9600
9991    258.5760
9992     29.6000
9993    243.1600
Length: 9994, dtype: float64

### Joins

There are scenarios when the data is not contained in a single file. Its not unusual to find data spread across many files. When that happens, we need some mechanism of joining different tables.

In [12]:
customers = pd.read_csv("customers.csv")
accounts = pd.read_csv("accounts.csv")

In [13]:
customers.head()

,Cust_id,Age
0,AA1,17
1,AA2,18
2,AA3,33
3,AA4,21
4,AA5,14


In [14]:
accounts.head()

,CustID,Account Type
0,AA1,AAA
1,AA6,AA
2,AA4,B
3,AA7,CCC
4,AA12,AAA


### Inner Join

- Join two tables based on common rows of a key column

In [15]:
pd.merge(customers,accounts,how='inner',left_on="Cust_id",right_on="CustID")

,Cust_id,Age,CustID,Account Type
0,AA1,17,AA1,AAA
1,AA4,21,AA4,B
2,AA6,81,AA6,AA


### Left Outer Join:
- Retain all the rows in the left table and give the matching rows in right table

In [16]:
pd.merge(customers,accounts,how='left',left_on="Cust_id",right_on="CustID")

,Cust_id,Age,CustID,Account Type
0,AA1,17,AA1,AAA
1,AA2,18,NaN,NaN
2,AA3,33,NaN,NaN
3,AA4,21,AA4,B
4,AA5,14,NaN,NaN
5,AA6,81,AA6,AA


### Right Outer Join:
- Retain all the rows in the right table and give the matching rows in left table

In [17]:
pd.merge(customers,accounts,how='right',left_on="Cust_id",right_on="CustID")

,Cust_id,Age,CustID,Account Type
0,AA1,17.0,AA1,AAA
1,AA6,81.0,AA6,AA
2,AA4,21.0,AA4,B
3,NaN,NaN,AA7,CCC
4,NaN,NaN,AA12,AAA
5,NaN,NaN,AA10,DDD


### Outer Join
- Join both the tables irrespective of any match in the key columns

In [18]:
pd.merge(customers,accounts,how='outer',left_on="Cust_id",right_on="CustID")

,Cust_id,Age,CustID,Account Type
0,AA1,17.0,AA1,AAA
1,AA2,18.0,NaN,NaN
2,AA3,33.0,NaN,NaN
3,AA4,21.0,AA4,B
4,AA5,14.0,NaN,NaN
5,AA6,81.0,AA6,AA
6,NaN,NaN,AA7,CCC
7,NaN,NaN,AA12,AAA
8,NaN,NaN,AA10,DDD


### Class Excercise (Joins)

- Use the files contributions.csv and candidates.csv. The file contributions.csv contains data on contributions made to political parties. The file candidates.csv contains data on the demographics of candidates belonging to different political parties. What was the highest contribution made on a Sunday?

- Use the files contributions.csv and candidates.csv. The file contributions.csv contains data on contributions made to political parties. The file candidates.csv contains data on the demographics of candidates belonging to different political parties. Is there a difference between the average donations received by Democrats on weekdays vs weekends? (In the column party, R stands for Republican and D stands for Democrats)

- Use the files contributions.csv and candidates.csv. The file contributions.csv contains data on contributions made to political parties. The file candidates.csv contains data on the demographics of candidates belonging to different political parties. The highest amount contributed on weekdays towards Democrats is?  (In the column party, R stands for Republican and D stands for Democrats)